# Introduccion

La encuesta nacional de programas presupuestales permita procesar información de diferentes programas presupuestales que se vienen implementando a nivel nacional; y servir de base para el diseño y orientación de políticas públicas para el mejoramiento de las condiciones de vida de la población.


Para este trabajo, se emplearán 2 datasets asociados a 2 módulos de la encuesta:
* 1. Características de la vivienda y el hogar
* 2. Características de los residentes habituales del hogar
 
**El obtetivo se centrará en análizar las variables más importantes que permitan conocer la situación de los servicios de agua y alcantarillado**

* Cobertura de agua y alcantarillado
* Cloración
* Continudad del servicio

# Análisis exploratorio

In [2]:
# Importante librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") #Ignorando los warnings

In [140]:
# Cargando datasets
bd_hogar = pd.read_csv('./data/CAP_100_URBANO_RURAL_3.csv',encoding='latin-1',sep=';')
bd_poblacion = pd.read_csv('./data/CAP_200_URBANO_RURAL_4.csv',encoding='latin-1',sep=';')

In [141]:
# Dimensión de la data
print("Número de hogares: "+str(bd_hogar.shape[0]))
print("Número de residentes: "+str(bd_poblacion.shape[0]))

Número de hogares: 42000
Número de residentes: 147007


In [142]:
# Visualización de registro de la base de hogares
bd_hogar.head()

,ANIO,MES,CONGLOMERADO,VIVIENDA,HOGAR,PER,NSELV,TSELV,VIVREM,NUMVIVREM,...,P189_3,P189_4,P189_5,P189_6,P189_7,P189_8,P189_8_O,REGIONNATU,ESTRATO,FACTOR
0,2023,1,55,2,1,1,32,1,2,,...,,,,,,,,1,2,"76,1065448790511"
1,2023,1,55,3,1,1,56,1,2,,...,,,,,,,,1,2,"76,1065448790511"
2,2023,1,55,6,1,1,102,1,2,,...,,,,,,,,1,2,"76,1065448790511"
3,2023,1,55,10,1,1,77,1,1,15,...,,,,,,,,1,2,"76,1065448790511"
4,2023,1,55,11,1,1,114,1,1,90,...,,,,,,,,1,2,"76,1065448790511"


In [11]:
# Visualización de registro de la base de población
bd_poblacion.head()

,ANIO,MES,CONGLOMERADO,VIVIENDA,HOGAR,P201,PER,NSELV,TSELV,VIVREM,...,P206,P207,P208_A,P208_B,P211A,P217A,P218,REGIONNATU,ESTRATO,FACTOR
0,2023,1,55,2,1,1,1,32,1,2,...,,1,52,,3,0,0,1,2,"76,1065448790511"
1,2023,1,55,2,1,2,1,32,1,2,...,,2,47,,3,0,0,1,2,"76,1065448790511"
2,2023,1,55,2,1,3,1,32,1,2,...,,2,9,,,,,1,2,"76,1065448790511"
3,2023,1,55,2,1,4,1,32,1,2,...,,1,24,,4,1,1,1,2,"76,1065448790511"
4,2023,1,55,2,1,5,1,32,1,2,...,,2,2,,,,,1,2,"76,1065448790511"


In [143]:
# Tranformación y filtro de datos (BD Poblacion)

# - Variables
# P204: ¿Es miembro del hogar?     1: Si, 2: No
# P205: ¿Se encuentra ausente del hogar 6 meses o más?    1: Si, 2: No
# P206: ¿Está presenten en el hogar 6 meses o más?   1: Si, 2: No

# - Tipos de datos
print(bd_poblacion[['P204','P205','P206']].dtypes)
print("----------")
bd_poblacion_completa = bd_poblacion[((bd_poblacion.P204 == 1) & (bd_poblacion.P205 == '2')) |
 ((bd_poblacion.P204 == 2) & (bd_poblacion.P206 == '1'))]

 # El factor de expanción tiene un formato errónes y por eso es un tipo de dato 'object'
print(bd_poblacion_completa.FACTOR.dtypes)
print("----------")
print(bd_poblacion_completa.FACTOR[:5])

print("----------")
bd_poblacion_completa['FACTOR'] = bd_poblacion_completa['FACTOR'].str.replace(",",".").astype(float)
print(bd_poblacion_completa.FACTOR[:5])

# Seleccionando las columnas finales
bd_poblacion_completa = bd_poblacion_completa[['CONGLOMERADO','VIVIENDA','HOGAR','PER','MES','P201']]

P204     int64
P205    object
P206    object
dtype: object
----------
object
----------
0    76,1065448790511
1    76,1065448790511
2    76,1065448790511
3    76,1065448790511
4    76,1065448790511
Name: FACTOR, dtype: object
----------
0    76.106545
1    76.106545
2    76.106545
3    76.106545
4    76.106545
Name: FACTOR, dtype: float64


In [163]:
# Tranformación y filtro de datos (BD Hogar)

# - Acceso a agua por red pública  (1: Con acceso    2: Sin acceso)
bd_hogar['AGUA_RPA'] = bd_hogar["P129G"].apply(lambda x: 1 if x<=3
                                  else 2 if x!=1
                                  else None)
# - Acceso a alcantaillado por red pública  (1: Con acceso    2: Sin acceso)
bd_hogar['ALCA_RPA'] = bd_hogar["P142A"].apply(lambda x: 1 if x<=2
                                  else 2 if x!=1
                                  else None)

# - Cloro residual (ejm: 0.05)
bd_hogar['CLORO_RESIDUAL'] = bd_hogar['P129D_ENT'].replace(' ',np.nan).astype(float) + \
    bd_hogar['P129D_DEC'].replace(' ',np.nan).astype(float)*0.1

# - Cloro adecuado (1: Cloro adecuado   2: Cloro inadecuado)
bd_hogar['NIVEL_CLORO_RESIDUAL'] = bd_hogar["CLORO_RESIDUAL"].apply(lambda x: 1 if x>=0.5
                                  else 2 if x<0.5
                                  else None)

# - Renombrando columnas
bd_hogar.rename(columns={'P130Z':'AGUA_TODOS_LOS_DIAS','P130ZA':'NUM_HORAS'},inplace=True)

bd_hogar_completa = bd_hogar[['PER','MES','CONGLOMERADO','VIVIENDA','HOGAR','AREA','NOMBREDD','REGIONNATU','ESTRATO'
                              ,'FACTOR','AGUA_TODOS_LOS_DIAS','NUM_HORAS','AGUA_RPA','ALCA_RPA',
                              'CLORO_RESIDUAL','NIVEL_CLORO_RESIDUAL']]

bd_hogar_completa['FACTOR'] = bd_hogar_completa['FACTOR'].str.replace(",",".").astype(float)



In [164]:
bd_enapres_completa = pd.merge(bd_poblacion_completa,bd_hogar_completa,
                               on=["CONGLOMERADO","VIVIENDA","HOGAR","PER","MES"],how="inner")

In [165]:
bd_enapres_completa

,CONGLOMERADO,VIVIENDA,HOGAR,PER,MES,P201,AREA,NOMBREDD,REGIONNATU,ESTRATO,FACTOR,AGUA_TODOS_LOS_DIAS,NUM_HORAS,AGUA_RPA,ALCA_RPA,CLORO_RESIDUAL,NIVEL_CLORO_RESIDUAL
0,55,2,1,1,1,1,1,TUMBES,1,2,76.106545,1,24,1,1,0.6,1.0
1,55,2,1,1,1,2,1,TUMBES,1,2,76.106545,1,24,1,1,0.6,1.0
2,55,2,1,1,1,3,1,TUMBES,1,2,76.106545,1,24,1,1,0.6,1.0
3,55,2,1,1,1,4,1,TUMBES,1,2,76.106545,1,24,1,1,0.6,1.0
4,55,2,1,1,1,5,1,TUMBES,1,2,76.106545,1,24,1,1,0.6,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140756,51561,25,1,3,12,1,2,TACNA,1,,27.988739,,,2,2,0.0,2.0
140757,51561,25,1,3,12,2,2,TACNA,1,,27.988739,,,2,2,0.0,2.0
140758,51561,25,1,3,12,3,2,TACNA,1,,27.988739,,,2,2,0.0,2.0
140759,51561,25,1,3,12,4,2,TACNA,1,,27.988739,,,2,2,0.0,2.0
